In [1]:
import pandas as pd
import numpy as np
import os
import nltools as nlt
import nilearn as nil
import nibabel as nib
import warnings
import glob
import random
import pickle
import dev_wtp_io_utils
import gc #garbage collection
from nilearn import plotting
from dev_wtp_io_utils import cv_train_test_sets, asizeof_fmt
from sklearn.model_selection import KFold,GroupKFold,LeaveOneOut,LeaveOneGroupOut
import os, warnings



#custom thing I have set in my jupyter notebook task.
cpus_available = int(os.getenv('CPUS_PER_TASK'))
print(cpus_available)

pd.set_option('display.max_rows', 99)


test_train_set = pd.read_csv("../data/train_test_markers_20210601T183243.csv")

with open('../data/Brain_Data_2sns_60subs.pkl', 'rb') as pkl_file:
    Brain_Data_allsubs = pickle.load(pkl_file)
    
dev_wtp_io_utils.check_BD_against_test_train_set(Brain_Data_allsubs,test_train_set)

#################################################
#######PRE-PROCESS

Brain_Data_allsubs.Y = Brain_Data_allsubs.X.response.copy()
print(Brain_Data_allsubs.Y.value_counts())
Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None
print(Brain_Data_allsubs.Y.value_counts())

import sys
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:60]:
    print(name + ': ' + str(size))
print(Brain_Data_allsubs.Y.isnull().value_counts())
Brain_Data_allsubs_nn = Brain_Data_allsubs[Brain_Data_allsubs.Y.isnull()==False]
print(len(Brain_Data_allsubs_nn))
print(len(Brain_Data_allsubs))


all_subs_nn_nifti = Brain_Data_allsubs_nn.to_nifti()
all_subs_nn_nifti_Y = Brain_Data_allsubs_nn.Y
all_subs_nn_nifti_groups = Brain_Data_allsubs_nn.X.subject
all_subs_nn_nifti_groups


all_subs_nn_nifti_metadata = Brain_Data_allsubs_nn.X


#################################################
#######GET SUB-SET


del Brain_Data_allsubs
#del Brain_Data_allsubs_grouped
gc.collect()

from nilearn.decoding import DecoderRegressor
dRegressor = DecoderRegressor(estimator = 'ridge_regressor', standardize= True,scoring="r2")


asizeof_fmt(Brain_Data_allsubs_nn)

asizeof_fmt(all_subs_nn_nifti)

print("cleaning")
train_y=all_subs_nn_nifti_Y
train_X = nil.image.clean_img(all_subs_nn_nifti,detrend=False,standardize=True)
train_groups = all_subs_nn_nifti_groups

del Brain_Data_allsubs_nn
gc.collect()

first_subs_nifti_metadata = all_subs_nn_nifti_metadata

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


4
checked for intersection and no intersection between the brain data and the subjects was found.
there were 60 subjects overlapping between the subjects marked for train data and the training dump file itself.
5.0    1164
6.0    1018
7.0     904
8.0     604
Name: response, dtype: int64
5.0    1164
6.0    1018
7.0     904
8.0     604
Name: response, dtype: int64
test_train_set: 9549
_i1: 2434
KFold: 1064
GroupKFold: 1064
LeaveOneOut: 1064
LeaveOneGroupOut: 1064
_oh: 232
Out: 232
pkl_file: 168
cv_train_test_sets: 136
asizeof_fmt: 136
__doc__: 113
_ih: 96
In: 96
__builtin__: 72
__builtins__: 72
pd: 72
np: 72
os: 72
nlt: 72
nil: 72
nib: 72
warnings: 72
glob: 72
random: 72
pickle: 72
dev_wtp_io_utils: 72
gc: 72
plotting: 72
sys: 72
_dh: 64
get_ipython: 64
__name__: 57
_: 49
__: 49
___: 49
_i: 49
_ii: 49
_iii: 49
exit: 48
quit: 48
Brain_Data_allsubs: 48
cpus_available: 28
__package__: 16
__loader__: 16
__spec__: 16
False    3690
True      150
Name: response, dtype: int64
3690
3840
cleaning


In [2]:

import pickle
with open("../data/first_subs_nifti_metadata.pkl", 'wb') as handle:
    pickle.dump(first_subs_nifti_metadata,handle)